In [1]:
from google.colab import drive
import sys

drive.mount('/content/drive')
module_path = '/content/gdrive/MyDrive/antenna-performance-prediction/antenna-performance-prediction/mylibrary/'
sys.path.insert(0, module_path)
print(sys.path)
!ls /content/drive/MyDrive/antenna-performance-prediction/antenna-performance-prediction/mylibrary/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['/content/gdrive/MyDrive/antenna-performance-prediction/antenna-performance-prediction/mylibrary/', '', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
manipulation.py  modeling.py  __pycache__  visualization.py


In [2]:
import pandas as pd
from mylibrary import manipulation
from mylibrary import visualization as visual


# Local execute.
origin_train = pd.read_csv('dataset/train.csv')
origin_test = pd.read_csv('dataset/test.csv')
x_info = pd.read_csv('dataset/meta/x_feature_info.csv')
y_info = pd.read_csv('dataset/meta/y_feature_info.csv')
submission = pd.read_csv("dataset/sample_submission.csv")

# Colab execute.
origin_train = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
origin_test = pd.read_csv('/content/drive/MyDrive/dataset/test.csv')
x_info = pd.read_csv('/content/drive/MyDrive/dataset/meta/x_feature_info.csv')
y_info = pd.read_csv('/content/drive/MyDrive/dataset/meta/y_feature_info.csv')
submission = pd.read_csv("/content/drive/MyDrive/dataset/sample_submission.csv")


x_train = origin_train.filter(regex='X')
y_train = origin_train.filter(regex='Y')

x_test = origin_test.drop(columns=['ID'])

x_name = manipulation.make_namegroup(x_train, x_info)
y_name = manipulation.make_namegroup(y_train, y_info)

visual.print_namegroup(x_name)
print()
visual.print_namegroup(y_name)

ModuleNotFoundError: ignored

In [ ]:
# 모든 값이 1이므로 사용하지 않음.
x_train = x_train.drop(columns=x_name['n차 검사 통과 여부'])
x_test = x_test.drop(columns=x_name['n차 검사 통과 여부'])


# 

In [ ]:
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor


xgb_model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.1, max_depth=7)
model = MultiOutputRegressor(xgb_model)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

In [ ]:
visual.pred_visualize(origin_train.filter(regex='Y'), y_pred)